In [ ]:
%%writefile main.nf
#!/usr/bin/env nextflow

process s3_workflow {
    
    label 's3'
    
    output:
    stdout emit: s3_download_log
    path('*.csv'), emit: csv_files
    
    script:
    """
    python3 /utils/s3_download_and_model.py --bucket-name placeholder --key-id placeholder --secret-key placeholder
    
    """
}

workflow {
    s3_workflow()
}

In [ ]:
%%writefile nextflow.config

process {
    withLabel: s3 {
        executor = 'awsbatch'
        queue = 'placeholder'
        container = 'placeholder'
    } 
}

aws {
    region = 'us-east-1'
    batch {
        cliPath = '/home/ec2-user/miniconda/bin/aws'
        jobRole = 'placeholder'
    }
}
workDir = 'placeholder'

docker.enabled = true

In [ ]:
!nextflow run main.nf -dsl2

## Download Workflow Results

In [ ]:
!pip install -q awscli
import os
import pandas as pd

In [ ]:
# Get the aws endpoints for each batch process. Since we are running two processes on 5 threads there will be 10 total endpoints.
end_points = []
with open(".nextflow.log", 'r') as f:
    for line in f:
        if "COMPLETED" in line:
            end_points.append(line.split(' ')[-1][:-2])

for i in range(len(end_points)):
    command = f'aws s3 cp {end_points[i]}/ ./results/ --recursive --exclude "*" --include "*" --quiet'
    os.system(command)

In [ ]:
results_df = pd.read_csv('results/rfc_results_df.csv')
results_df.head()